In [26]:
import cv2 as cv
import numpy as np
import torch
import matplotlib.pyplot as plt
import uuid  # unique identifier
import os
import time
import mediapipe as mp
import time

## Training from Scratch (simple object detection)

In [27]:
IMAGES_PATH = os.path.join('data_rps', 'images_rps') # data/images
labels = ['Rock', 'Paper', 'Scissors', 'Lizard', 'Spock']
num_imgs = 100

### Collecting Images

In [7]:
cap = cv.VideoCapture(0)
for label in labels:
    print(f"Collecting images for {label}")
    time.sleep(5)
    for img_num in range(num_imgs):
        ret, frame = cap.read()
        print(img_num, end=' ')
        if ret:
            cv.imshow('Camera', frame)
            #New Path
            imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1()) + '.jpg')
            print(imgname)
            # Writing out
            cv.imwrite(imgname, frame)
            time.sleep(1)

KeyboardInterrupt: 

### Create a new dataset!!! (use labelImg, .yaml file)

In [ ]:
!python3 train.py --img 320 --batch 16 --epochs 500 --data dataset.yaml --weights yolov5m.pt --workers 2

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp7/weights/last.pt', force_reload = True)
cap = cv.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        results = model(frame)
        cv.imshow('YOLO', np.squeeze(results.render()))

        if cv.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv.destroyAllWindows()

### Overcomplicating it with HandLandmarks

In [29]:
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils
cap = cv.VideoCapture(0)
cTime = pTime = 0
while cap.isOpened():
    ret, img = cap.read()
    if ret:
        h, w, c = img.shape
        rbgimg = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        res = hands.process(rbgimg)
        if res.multi_hand_landmarks:
            for handLms in res.multi_hand_landmarks:
                for i, lm in enumerate(handLms.landmark):
                    cx = int(w * lm.x)
                    cy = int(h * lm.y)
                    if i and not i%4 :
                        cv.circle(img, (cx, cy), 30, (0, 0, 255), -1)
                rx = int(w * (handLms.landmark[5].x  + handLms.landmark[0].x) / 2)
                ry = int(h * (handLms.landmark[5].y  + handLms.landmark[0].y) / 2)
                cv.circle(img, (rx, ry), 150, (255, 255, 0), -1)
                mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
        
        cTime = time.time()
        fps = 1/(cTime - pTime)
        pTime = cTime
        cv.putText(img, str(int(fps)), (10, 70), cv.FONT_HERSHEY_PLAIN, 3, (255, 255, 0), 3)
        cv.imshow('Hands', img)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    
    
cap.release()
cv.destroyAllWindows()
    
    
    